# Downloading Price Data

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import re

In [2]:
# Set data directory path
DATA_DIR = Path("../data/finra_clean")

In [3]:
# Get all CSV files matching the pattern
csv_files = sorted(DATA_DIR.glob("shrt*_clean.csv"))

print(f"Found {len(csv_files)} files in {DATA_DIR}")
if csv_files:
    print(f"First file: {csv_files[0].name}")
    print(f"Last file: {csv_files[-1].name}")

Found 182 files in ../data/finra_clean
First file: shrt20171229_clean.csv
Last file: shrt20250715_clean.csv


In [4]:
# Load all files into a single dataframe
dfs = []

for file in csv_files:
    df = pd.read_csv(file)
    # Extract date from filename (shrtYYYYMMDD_clean.csv)
    date_match = re.search(r'shrt(\d{8})_clean', file.name)
    if date_match:
        date_str = date_match.group(1)
        df['file_date'] = pd.to_datetime(date_str, format='%Y%m%d')
    
    dfs.append(df)
    print(f"Loaded {file.name}: {len(df)} rows")

# Combine all dataframes
data = pd.concat(dfs, ignore_index=True)

print(f"\nTotal rows loaded: {len(data):,}")
print(f"Columns: {list(data.columns)}")

Loaded shrt20171229_clean.csv: 15495 rows
Loaded shrt20180112_clean.csv: 15579 rows
Loaded shrt20180131_clean.csv: 15627 rows
Loaded shrt20180215_clean.csv: 15561 rows
Loaded shrt20180228_clean.csv: 15628 rows
Loaded shrt20180315_clean.csv: 15486 rows
Loaded shrt20180329_clean.csv: 15398 rows
Loaded shrt20180413_clean.csv: 15378 rows
Loaded shrt20180430_clean.csv: 15474 rows
Loaded shrt20180515_clean.csv: 15533 rows
Loaded shrt20180531_clean.csv: 15498 rows
Loaded shrt20180615_clean.csv: 15563 rows
Loaded shrt20180629_clean.csv: 15573 rows
Loaded shrt20180713_clean.csv: 15702 rows
Loaded shrt20180731_clean.csv: 15783 rows
Loaded shrt20180815_clean.csv: 15744 rows
Loaded shrt20180831_clean.csv: 15749 rows
Loaded shrt20180914_clean.csv: 15775 rows
Loaded shrt20180928_clean.csv: 15873 rows
Loaded shrt20181015_clean.csv: 15905 rows
Loaded shrt20181031_clean.csv: 15825 rows
Loaded shrt20181115_clean.csv: 15802 rows
Loaded shrt20181130_clean.csv: 15691 rows
Loaded shrt20181214_clean.csv: 157

In [5]:
print(f"{data.symbolCode.unique()}")
print(f"Number of unique symbols: {len(data.symbolCode.unique())}")
print(f"Number of CSV files: {len(csv_files)}")

# Count occurrences of each symbol (much faster than filtering repeatedly)
symbol_counts = data.symbolCode.value_counts()

# Filter symbols that appear exactly len(csv_files) times
symbols_to_use = symbol_counts[symbol_counts == len(csv_files)].index.tolist()

print(f"Symbols with complete data: {len(symbols_to_use)}")
print(symbols_to_use)

['A' 'AA' 'AAALF' ... 'WJLTF' 'XOEF' 'YOUL']
Number of unique symbols: 44674
Number of CSV files: 182
Symbols with complete data: 5908
['A', 'KMTUF', 'KNYJF', 'KNX', 'KNSL', 'KNOP', 'KNNGF', 'KNKZF', 'KNDI', 'KNCRF', 'KNCAF', 'KNBWF', 'KNAMF', 'KN', 'KMX', 'KMTUY', 'KMT', 'KODK', 'KMRPF', 'KMRCF', 'KMPR', 'KMMPF', 'KMI', 'KMGIF', 'KMDA', 'KMB', 'KLPEF', 'KLKNF', 'KLIC', 'KLAC', 'KKR', 'KKPNF', 'KO', 'KOF', 'KKKUF', 'KRG', 'KTCC', 'KT', 'KSS', 'KSFTF', 'KSANF', 'KSA', 'KRYS', 'KRYAY', 'KRYAF', 'KRP', 'KRO', 'KRNY', 'KRNT', 'KRMA', 'KREVF', 'KOLD', 'KREF', 'KRE', 'KRDXF', 'KRC', 'KR', 'KPTI', 'KPLUF', 'KPELF', 'KOSS', 'KOSCF', 'KOS', 'KORU', 'KOPN', 'KOP', 'KKOYF', 'KIRK', 'KTOS', 'KBH', 'KDDIY', 'KDDIF', 'KCLI', 'KCE', 'KCDMF', 'KBYPF', 'KBWY', 'KBWR', 'KBWP', 'KBWD', 'KBWB', 'KBSTF', 'KBRLF', 'KBR', 'KBE', 'KELTF', 'KBCSF', 'KBAGF', 'KBA', 'KB', 'KAR', 'KAOCF', 'KANKF', 'KALV', 'KALU', 'KALA', 'KAKKF', 'KAJMF', 'KAI', 'KAEPF', 'KE', 'KELYA', 'KIO', 'KGFHY', 'KINUF', 'KINS', 'KIMPRM', '

In [6]:
from dotenv import load_dotenv
import requests
import os
import pandas as pd
from pathlib import Path
import time

# Load environment variables
load_dotenv()

# Load API key
API_KEY = os.getenv("ALPHAVANTAGE_API_KEY")

# Set price data directory path
PRICE_DIR = Path("../data/prices")
PRICE_DIR.mkdir(parents=True, exist_ok=True)

url = 'https://www.alphavantage.co/query'

# Store each symbol's data in a list first
all_data = []
failed_symbols = []
api_limit_reached = False

for i, symbol in enumerate(symbols_to_use):
    print(f"[{i+1}/{len(symbols_to_use)}] Downloading data for {symbol}...")
    
    # Define parameters for the API request
    params = {
        'function': 'TIME_SERIES_DAILY_ADJUSTED',
        'symbol': symbol,
        'apikey': API_KEY,
        'outputsize': 'full',
        'datatype': 'json'
    }
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        
        # Check for various error conditions
        if "Error Message" in data:
            print(f"Error for {symbol}: Invalid symbol or API error")
            failed_symbols.append({'symbol': symbol, 'reason': 'Invalid symbol'})
            continue
            
        elif "Note" in data:
            print(f"API call frequency limit reached. Message: {data['Note']}")
            failed_symbols.append({'symbol': symbol, 'reason': 'API limit'})
            api_limit_reached = True
            print("  Waiting 60 seconds before continuing...")
            time.sleep(60)  # Wait a minute before trying again
            continue
            
        elif "Information" in data:
            print(f"API message: {data['Information']}")
            failed_symbols.append({'symbol': symbol, 'reason': 'API information message'})
            continue
            
        elif "Time Series (Daily)" not in data:
            print(f"No time series data found for {symbol}")
            failed_symbols.append({'symbol': symbol, 'reason': 'No data available'})
            continue
        
        # Convert to DataFrame and transpose
        df_temp = pd.DataFrame(data["Time Series (Daily)"]).T
        
        # Convert index to datetime
        df_temp.index = pd.to_datetime(df_temp.index)
        
        # Add symbol as a column level
        df_temp.columns = pd.MultiIndex.from_product([[symbol], df_temp.columns])
        
        all_data.append(df_temp)
        print(f"Successfully downloaded {symbol}")
        
    except requests.exceptions.RequestException as e:
        print(f"Network error for {symbol}: {e}")
        failed_symbols.append({'symbol': symbol, 'reason': f'Network error: {str(e)}'})
        continue
        
    except Exception as e:
        print(f"Unexpected error for {symbol}: {e}")
        failed_symbols.append({'symbol': symbol, 'reason': f'Unexpected error: {str(e)}'})
        continue
    
    # Add a small delay
    time.sleep(0.01)  
    
    # Progress report every 25 symbols
    if (i + 1) % 25 == 0:
        print(f"\nProgress Report: {i+1}/{len(symbols_to_use)} completed")
        print(f"   Successfully downloaded: {len(all_data)}")
        print(f"   Failed: {len(failed_symbols)}\n")

# Combine all successful dataframes
if all_data:
    price_df = pd.concat(all_data, axis=1)
    price_df = price_df.sort_index()
    price_df = price_df.astype(float)
    
    print("\n" + "="*50)
    print(f"Download Complete!")
    print(f"   Total symbols requested: {len(symbols_to_use)}")
    print(f"   Successfully downloaded: {len(all_data)}")
    print(f"   Failed: {len(failed_symbols)}")
    print(f"   Data shape: {price_df.shape}")
    print(f"   Date range: {price_df.index.min()} to {price_df.index.max()}")
    print("="*50)
    
    # Save the successful data
    from datetime import datetime
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_file = PRICE_DIR / f"price_data_multiindex_{timestamp}.xlsx"
    
    print(f"\nSaving data to {output_file}...")
    price_df.to_excel(output_file, sheet_name='All_Price_Data')
    print(f"   Data saved successfully!")
    
else:
    print("\nNo data was successfully downloaded")
    price_df = pd.DataFrame()

# Save failed symbols report
if failed_symbols:
    failed_df = pd.DataFrame(failed_symbols)
    failed_file = PRICE_DIR / f"failed_symbols_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    failed_df.to_csv(failed_file, index=False)
    
    print(f"\nFailed symbols report saved to: {failed_file}")
    print("\nFailed symbols summary:")
    print(failed_df['reason'].value_counts())
    
    # Show first few failed symbols
    print("\nFirst 10 failed symbols:")
    print(failed_df.head(10).to_string())

[1/5908] Downloading data for A...
Successfully downloaded A
[2/5908] Downloading data for KMTUF...
Successfully downloaded KMTUF
[3/5908] Downloading data for KNYJF...
Successfully downloaded KNYJF
[4/5908] Downloading data for KNX...
Successfully downloaded KNX
[5/5908] Downloading data for KNSL...
Successfully downloaded KNSL
[6/5908] Downloading data for KNOP...
Successfully downloaded KNOP
[7/5908] Downloading data for KNNGF...
Successfully downloaded KNNGF
[8/5908] Downloading data for KNKZF...
Successfully downloaded KNKZF
[9/5908] Downloading data for KNDI...
Successfully downloaded KNDI
[10/5908] Downloading data for KNCRF...
Successfully downloaded KNCRF
[11/5908] Downloading data for KNCAF...
Successfully downloaded KNCAF
[12/5908] Downloading data for KNBWF...
Successfully downloaded KNBWF
[13/5908] Downloading data for KNAMF...
Successfully downloaded KNAMF
[14/5908] Downloading data for KN...
Successfully downloaded KN
[15/5908] Downloading data for KMX...
Successfully do

ValueError: This sheet is too large! Your sheet size is: 6499, 46544 Max sheet size is: 1048576, 16384

In [7]:
price_df

A                                                          \
           1. open   2. high  3. low 4. close 5. adjusted close  6. volume   
1999-11-01     NaN       NaN     NaN      NaN               NaN        NaN   
1999-11-02     NaN       NaN     NaN      NaN               NaN        NaN   
1999-11-03     NaN       NaN     NaN      NaN               NaN        NaN   
1999-11-04     NaN       NaN     NaN      NaN               NaN        NaN   
1999-11-05     NaN       NaN     NaN      NaN               NaN        NaN   
...            ...       ...     ...      ...               ...        ...   
2025-08-27  118.50  119.2000  117.51   118.69            118.69  1698630.0   
2025-08-28  118.80  125.4204  118.20   124.96            124.96  3886441.0   
2025-08-29  125.00  126.3000  124.11   125.66            125.66  1781215.0   
2025-09-02  124.88  125.5200  121.80   125.21            125.21  2412797.0   
2025-09-03  124.96  125.8300  123.72   125.22            125.22  2496977.0   

                                                     KMTUF          ...  \
           7. dividend amount 8. split coefficient 1. open 2. high  ...   
1999-11-01                NaN                  NaN     NaN     NaN  ...   
1999-11-02                NaN                  NaN     NaN     NaN  ...   
1999-11-03                NaN                  NaN     NaN     NaN  ...   
1999-11-04                NaN                  NaN     NaN     NaN  ...   
1999-11-05                NaN                  NaN     NaN     NaN  ...   
...                       ...                  ...     ...     ...  ...   
2025-08-27                0.0                  1.0  33.663   34.35  ...   
2025-08-28                0.0                  1.0  36.000   36.00  ...   
2025-08-29                0.0                  1.0  33.750   34.73  ...   
2025-09-02                0.0                  1.0  34.730   34.73  ...   
2025-09-03                0.0                  1.0  32.990   32.99  ...   

                          BST                          EGP                    \
           7. dividend amount 8. split coefficient 1. open  2. high   3. low   
1999-11-01                NaN                  NaN   18.38   18.380   18.130   
1999-11-02                NaN                  NaN   18.25   18.380   18.250   
1999-11-03                NaN                  NaN   18.38   18.500   18.190   
1999-11-04                NaN                  NaN   18.50   18.560   18.310   
1999-11-05                NaN                  NaN   18.38   18.440   18.310   
...                       ...                  ...     ...      ...      ...   
2025-08-27                0.0                  1.0  168.00  171.245  167.845   
2025-08-28                0.0                  1.0  168.89  168.890  166.496   
2025-08-29                0.0                  1.0  168.59  170.215  168.305   
2025-09-02                0.0                  1.0  167.63  167.895  165.215   
2025-09-03                0.0                  1.0  165.60  166.985  165.420   

                                                                    \
           4. close 5. adjusted close 6. volume 7. dividend amount   
1999-11-01    18.13          5.769661   10400.0                0.0   
1999-11-02    18.38          5.849221   15600.0                0.0   
1999-11-03    18.50          5.887409   14000.0                0.0   
1999-11-04    18.38          5.849221   20000.0                0.0   
1999-11-05    18.31          5.826944   23400.0                0.0   
...             ...               ...       ...                ...   
2025-08-27   168.77        168.770000  451866.0                0.0   
2025-08-28   168.34        168.340000  305432.0                0.0   
2025-08-29   169.56        169.560000  347683.0                0.0   
2025-09-02   165.85        165.850000  290440.0                0.0   
2025-09-03   166.09        166.090000  282002.0                0.0   

                                 
           8. split coefficient  
1999-1

In [8]:
# Combine all successful dataframes
if all_data:
    price_df = pd.concat(all_data, axis=1)
    price_df = price_df.sort_index()
    price_df = price_df.astype(float)
    
    print("\n" + "="*50)
    print(f"Download Complete!")
    print(f"   Total symbols requested: {len(symbols_to_use)}")
    print(f"   Successfully downloaded: {len(all_data)}")
    print(f"   Failed: {len(failed_symbols)}")
    print(f"   Data shape: {price_df.shape}")
    print(f"   Date range: {price_df.index.min()} to {price_df.index.max()}")
    print("="*50)
    
    # Save the successful data as Parquet
    from datetime import datetime
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_file = PRICE_DIR / f"price_data_multiindex_{timestamp}.parquet"
    
    print(f"\nSaving data to {output_file}...")
    price_df.to_parquet(output_file, engine='pyarrow')  # or engine='fastparquet'
    
    # Calculate and display file size
    file_size_mb = output_file.stat().st_size / (1024**2)
    print(f"Data saved successfully!")
    print(f"File size: {file_size_mb:.2f} MB")
    print(f"Compression ratio: {(price_df.memory_usage(deep=True).sum() / (1024**2)) / file_size_mb:.1f}x")
    
    # Also save a small Excel sample for quick viewing (first 100 symbols, all dates)
    sample_symbols = price_df.columns.get_level_values(0).unique()[:100]
    sample_df = price_df[sample_symbols]
    sample_file = PRICE_DIR / f"price_data_sample_{timestamp}.xlsx"
    sample_df.to_excel(sample_file, sheet_name='Sample_Data')
    print(f"Sample Excel saved (first 100 symbols): {sample_file.name}")
    
else:
    print("\nNo data was successfully downloaded")
    price_df = pd.DataFrame()

# Save failed symbols report
if failed_symbols:
    failed_df = pd.DataFrame(failed_symbols)
    failed_file = PRICE_DIR / f"failed_symbols_{timestamp}.csv"
    failed_df.to_csv(failed_file, index=False)
    
    print(f"\nFailed symbols report saved to: {failed_file}")
    print("\nFailed symbols summary:")
    print(failed_df['reason'].value_counts())
    
    # Show first few failed symbols
    print("\nFirst 10 failed symbols:")
    print(failed_df.head(10).to_string())


Download Complete!
   Total symbols requested: 5908
   Successfully downloaded: 5818
   Failed: 90
   Data shape: (6499, 46544)
   Date range: 1999-11-01 00:00:00 to 2025-09-03 00:00:00

Saving data to ../data/prices/price_data_multiindex_20250904_113138.parquet...
Data saved successfully!
File size: 655.88 MB
Compression ratio: 3.5x
Sample Excel saved (first 100 symbols): price_data_sample_20250904_113138.xlsx

Failed symbols report saved to: ../data/prices/failed_symbols_20250904_113138.csv

Failed symbols summary:
reason
Invalid symbol                                                                                                 88
Network error: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))                    1
Network error: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))     1
Name: count, dtype: int64

First 10 failed symbols:
   symbol          reason
0  KIMPRM  Invalid symbol
1  KIMPRL  Invalid symb